In [1]:
!pip install datasets
!pip install --upgrade sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which

In [14]:
import pandas as pd
df=pd.read_csv(r'combined.csv')
df['translation'] = df.apply(lambda row: {'en': row['text_x'], 'ml': row['text_y']}, axis=1)
df = df.drop(columns=['text_x', 'text_y'])
data_list = df.to_dict(orient='records')
data_list=data_list[:8000]
from datasets import Dataset, DatasetDict
dataset = Dataset.from_list(data_list)
dataset = dataset.train_test_split(test_size=0.2)

In [15]:
from transformers import AutoTokenizer
checkpoint='google/mt5-small'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
source_lang='en'
target_lang='ml'
prefix='translate English to Malayalam:'
def preprocess_text(examples):
  inputs=[prefix+example[source_lang] for example in examples['translation']]
  targets=[example[target_lang] for example in examples['translation']]
  tokenized_inputs=tokenizer(inputs,text_target=targets,max_length=128,truncation=True)
  return tokenized_inputs
preprocessed_dataset=dataset.map(preprocess_text,batched=True)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [17]:
from transformers import DataCollatorForSeq2Seq
datacollator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=checkpoint)

In [18]:
from transformers import TFAutoModelForSeq2SeqLM
model=TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [19]:
train_set=model.prepare_tf_dataset(
    preprocessed_dataset['train'],
    batch_size=16,
    shuffle=True,
    collate_fn=datacollator
)

In [20]:
import tensorflow as tf
model.compile(tf.keras.optimizers.Adam(learning_rate=2e-5))

In [21]:
model.summary()

Model: "tfmt5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300176768 (1.12 GB)
Trainable params: 300176768 (1.12 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(train_set,epochs=5)

Epoch 1/5
400/400 [==============================] - 327s 598ms/step - loss: 17.1215
Epoch 2/5
400/400 [==============================] - 232s 581ms/step - loss: 9.7068
Epoch 3/5
400/400 [==============================] - 231s 578ms/step - loss: 8.1601
Epoch 4/5
400/400 [==============================] - 232s 579ms/step - loss: 7.4475
Epoch 5/5
400/400 [==============================] - 232s 579ms/step - loss: 7.0206
